In [25]:
import conveyal_vars
import json
from siuba import *

# Matching our warehouse to Conveyal's feed ids...

In [3]:
## can't query directly since Hub instance won't be authenticated...

# bundle_url = 'https://analysis.conveyal.com/api/db/bundles'

https://analysis.conveyal.com/api/db/bundles

In [8]:
with open('bundles.json') as file:
    bundles = json.load(file)

In [10]:
len(bundles)

24

In [13]:
conveyal_vars.target_date.isoformat()

'2023-10-18'

In [15]:
current_bundle = [bundle for bundle in bundles if bundle['name'] == conveyal_vars.target_date.isoformat()][0]

In [16]:
current_bundle

{'_id': '6541350ddad1a760f30e32be',
 'name': '2023-10-18',
 'nonce': '6541350ddad1a760f30e32bf',
 'createdAt': '2023-10-31T17:10:37.097Z',
 'updatedAt': '2023-10-31T17:10:37.097Z',
 'accessGroup': 'caltrans',
 'createdBy': 'eric.dasmalchi@dot.ca.gov',
 'updatedBy': 'eric.dasmalchi@dot.ca.gov',
 'regionId': '6352a1d5e1a8e551137301aa',
 'osmId': '6541350ddad1a760f30e32c0',
 'feedGroupId': '6541351adad1a760f30e32c1',
 'north': 54.77395978,
 'south': 24.552569,
 'east': -68.811782,
 'west': -124.498078553064,
 'serviceStart': '2013-01-01',
 'serviceEnd': '2029-12-31',
 'feeds': [{'feedId': '6541351adad1a760f30e32c2',
   'name': 'Martz Trailways, Executive Transportation, Route 66 Express, +45 more: 2023-10-17 to 2024-10-14',
   'bundleScopedFeedId': '6541351adad1a760f30e32c2_6541351adad1a760f30e32c1',
   'serviceStart': '2023-10-15',
   'serviceEnd': '2024-10-14',
   'checksum': 3610834750,
   'errors': [{'type': 'RangeError',
     'count': 48,
     'someErrors': [{'file': 'stop_times',
  

In [20]:
feeds = [
    {key:val for key, val in bundle_feed.items() if key != 'errors'}
    for bundle_feed in current_bundle['feeds']
]

In [21]:
#  many-agency feed is Amtrak (all the thruway operators...)
feeds

[{'feedId': '6541351adad1a760f30e32c2',
  'name': 'Martz Trailways, Executive Transportation, Route 66 Express, +45 more: 2023-10-17 to 2024-10-14',
  'bundleScopedFeedId': '6541351adad1a760f30e32c2_6541351adad1a760f30e32c1',
  'serviceStart': '2023-10-15',
  'serviceEnd': '2024-10-14',
  'checksum': 3610834750},
 {'feedId': '65413534dad1a760f30e32c3',
  'name': 'B-Line: 2023-08-20 to 2024-01-20',
  'bundleScopedFeedId': '65413534dad1a760f30e32c3_6541351adad1a760f30e32c1',
  'serviceStart': '2023-08-20',
  'serviceEnd': '2024-01-20',
  'checksum': 1559914972},
 {'feedId': '65413535dad1a760f30e32c4',
  'name': 'Capitol Corridor Joint Powers Authority: 2023-04-03 to 2023-12-31',
  'bundleScopedFeedId': '65413535dad1a760f30e32c4_6541351adad1a760f30e32c1',
  'serviceStart': '2023-04-03',
  'serviceEnd': '2023-12-31',
  'checksum': 1917332726},
 {'feedId': '65413536dad1a760f30e32c5',
  'name': 'currypublictransit-brookings-or-us: 2023-08-23 to 2023-11-01',
  'bundleScopedFeedId': '65413536d

In [22]:
feeds[2]

{'feedId': '65413535dad1a760f30e32c4',
 'name': 'Capitol Corridor Joint Powers Authority: 2023-04-03 to 2023-12-31',
 'bundleScopedFeedId': '65413535dad1a760f30e32c4_6541351adad1a760f30e32c1',
 'serviceStart': '2023-04-03',
 'serviceEnd': '2023-12-31',
 'checksum': 1917332726}

In [31]:
[feed for feed in feeds if feed['name'].startswith('Capitol Corridor')]

[{'feedId': '65413535dad1a760f30e32c4',
  'name': 'Capitol Corridor Joint Powers Authority: 2023-04-03 to 2023-12-31',
  'bundleScopedFeedId': '65413535dad1a760f30e32c4_6541351adad1a760f30e32c1',
  'serviceStart': '2023-04-03',
  'serviceEnd': '2023-12-31',
  'checksum': 1917332726}]

## Deriving start and end dates from one of our feeds?

In [23]:
import download_data

In [27]:
df = download_data.regions_and_feeds >> distinct(_.region, _.feed_key, _keep_all=True)

In [28]:
df >> head(3)

,region,feed_key,gtfs_dataset_name,base64_url,date
0,norcal,7bd3d8c32eda4869c4d7f8bf2aec5bb0,Flixbus Schedule,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,2023-10-18
1,central,7bd3d8c32eda4869c4d7f8bf2aec5bb0,Flixbus Schedule,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,2023-10-18
2,socal,7bd3d8c32eda4869c4d7f8bf2aec5bb0,Flixbus Schedule,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,2023-10-18


In [30]:
df >> filter(_.gtfs_dataset_name.str.contains('Capitol'))

,region,feed_key,gtfs_dataset_name,base64_url,date
10,norcal,ba40a9f4ba1b8204a19ca2a5af293eb3,Bay Area 511 Capitol Corridor Schedule,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-10-18
11,central,ba40a9f4ba1b8204a19ca2a5af293eb3,Bay Area 511 Capitol Corridor Schedule,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,2023-10-18


In [32]:
from calitp_data_analysis.tables import tbls

In [34]:
cap_key = 'ba40a9f4ba1b8204a19ca2a5af293eb3'

In [35]:
tbls.mart_gtfs.dim_schedule_feeds() >> filter(_.key == cap_key)

,key,base64_url,download_success,unzip_success,zipfile_extract_md5hash,feed_timezone,_dt,_valid_from,_valid_to,_is_current
0,ba40a9f4ba1b8204a19ca2a5af293eb3,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,True,True,402626553e9a19dc34d0446f379f7c66,America/Los_Angeles,2023-08-17,2023-08-17 03:00:34.068721+00:00,2023-12-15 03:00:32.585748+00:00,False


In [41]:
#  gotta use calendar + calendar_dates here

# (tbls.mart_gtfs.fct_daily_schedule_feeds()
#  >> filter(_.feed_key == cap_key)
#  >> group_by(_.feed_key)
#  >> summarize(min_date = _.date.min(), max_date = _.date.max())
# )

In [39]:
(tbls.mart_gtfs.dim_calendar()
 >> filter(_.feed_key == cap_key)
 >> group_by(_.feed_key)
 >> summarize(min_date = _.start_date.min(), max_date = _.end_date.max())
)

,feed_key,min_date,max_date
0,ba40a9f4ba1b8204a19ca2a5af293eb3,2023-04-03,2023-12-31


In [40]:
(tbls.mart_gtfs.dim_calendar_dates()
 >> filter(_.feed_key.isin(df.feed_key))
 >> group_by(_.feed_key)
 >> summarize(min_date = _.date.min(), max_date = _.date.max())
)

,feed_key,min_date,max_date
0,ba40a9f4ba1b8204a19ca2a5af293eb3,2023-05-29,2023-12-25


In [89]:
filter_feeds = filter(_.feed_key.isin(norcal.feed_key))

In [90]:
dim_cd = (tbls.mart_gtfs.dim_calendar_dates()
          >> select(_.feed_key, _.date)
          >> filter_feeds
          >> group_by(_.feed_key)
          >> summarize(min_cd = _.date.min(), max_cd = _.date.max())
         )

dim_cal = (tbls.mart_gtfs.dim_calendar()
           >> select(_.feed_key, _.start_date, _.end_date)
           >> filter_feeds
           >> group_by(_.feed_key)
           >> summarize(min_cal = _.start_date.min(), max_cal = _.end_date.max())
          )

In [91]:
joined = (dim_cal
 >> full_join(_, dim_cd, on = 'feed_key')
 # >> collect()
         )

In [92]:
norcal.shape

(25, 5)

In [93]:
df2 = joined >> collect()

In [102]:
df2.min_cal = df2.min_cal.fillna(df2.min_cd)
df2.max_cal = df2.max_cal.fillna(df2.max_cd)
df2.min_cd = df2.min_cd.fillna(df2.min_cal)
df2.max_cd = df2.max_cd.fillna(df2.max_cal)

In [104]:
df2['min_combined'] = df2[['min_cal', 'max_cal', 'max_cd', 'min_cd']].values.min(axis=1)
df2['max_combined'] = df2[['min_cal', 'max_cal', 'max_cd', 'min_cd']].values.max(axis=1)

In [105]:
df2

,feed_key,min_cal,max_cal,max_cd,min_cd,min_combined,max_combined
0,2c81de89a7463d5ca5df230d0ceb40fd,2023-09-03,2024-06-30,2024-06-30,2023-09-03,2023-09-03,2024-06-30
1,f7a2121f01eef823fb9018028a5c7f41,2021-08-16,2024-06-01,2024-05-27,2023-11-11,2021-08-16,2024-06-01
2,9c7eaa3448667d131fc58bd307dbccbb,2022-07-01,2024-03-01,2024-03-01,2022-07-01,2022-07-01,2024-03-01
3,b4970d6cc7e206d9e667796130394790,2023-10-15,2024-10-14,2024-10-14,2023-10-15,2023-10-15,2024-10-14
4,7bd3d8c32eda4869c4d7f8bf2aec5bb0,2023-10-15,2025-01-16,2024-03-28,2023-10-17,2023-10-15,2025-01-16
5,102b5149d072a697aeda213b8b72811b,2023-08-14,2024-06-07,2024-05-27,2023-10-13,2023-08-14,2024-06-07
6,8ba295b256104176edafa35cec721b00,2023-09-07,2024-05-25,2024-02-19,2023-10-09,2023-09-07,2024-05-25
7,7b523354fae424de5845d3e5f12da59a,2022-01-22,2023-11-01,2023-09-04,2023-09-04,2022-01-22,2023-11-01
8,c036c6409639fec1cda2d17d18b317e4,2019-12-02,2024-02-01,2024-02-01,2019-12-02,2019-12-02,2024-02-01
9,748268ad11a5e83db5e4782e822e5998,2020-10-01,2024-02-01,2024-01-01,2023-11-23,2020-10-01,2024-02-01
